# Using the MANN Package to train a Convolutional Neural Network

In this notebook, the MANN package will be used to train pruned convolutional neural networks.  We will train two single-task networks on two separate tasks and one multitask network which performs both tasks.

In [1]:
# Load the MANN package and TensorFlow
import tensorflow as tf
import mann

In [2]:
# Load both MNIST tasks
(digit_x_train, digit_y_train), (digit_x_test, digit_y_test) = tf.keras.datasets.mnist.load_data()
(fashion_x_train, fashion_y_train), (fashion_x_test, fashion_y_test) = tf.keras.datasets.fashion_mnist.load_data()

# Reshape the x data so they have channels, divide by 255 to place all pixel values in [0, 1]
digit_x_train = digit_x_train.reshape(digit_x_train.shape + (1,))/255
digit_x_test = digit_x_test.reshape(digit_x_test.shape + (1,))/255
fashion_x_train = fashion_x_train.reshape(fashion_x_train.shape + (1,))/255
fashion_x_test = fashion_x_test.reshape(fashion_x_test.shape + (1,))/255

# Reshape the y data to have 1 column
digit_y_train = digit_y_train.reshape(-1, 1)
digit_y_test = digit_y_test.reshape(-1, 1)
fashion_y_train = fashion_y_train.reshape(-1, 1)
fashion_y_test = fashion_y_test.reshape(-1, 1)

# Create a callback for early stopping
callback = tf.keras.callbacks.EarlyStopping(min_delta = 0.01, patience = 3, restore_best_weights = True)

## Create the first model

This first model is a convolutional model which will perform MNIST digit recognition. It will be pruned utilizing the MANN package so that most of its weights are 0.

In [3]:
# Create the input layer for the digit task
input_layer = tf.keras.layers.Input(digit_x_train.shape[1:])

# Create the convolutional blocks
x = mann.layers.MaskedConv2D(
    filters = 32,
    kernel_size = 3,
    padding = 'same',
    strides = 1,
    activation = 'relu'
)(input_layer)
x = mann.layers.MaskedConv2D(
    filters = 32,
    kernel_size = 3,
    padding = 'same',
    strides = 1,
    activation = 'relu'
)(x)
x = tf.keras.layers.MaxPool2D(
    pool_size = 2,
    strides = 1,
    padding = 'valid'
)(x)
x = mann.layers.MaskedConv2D(
    filters = 64,
    kernel_size = 3,
    padding = 'same',
    strides = 1,
    activation = 'relu'
)(x)
x = mann.layers.MaskedConv2D(
    filters = 64,
    kernel_size = 3,
    padding = 'same',
    strides = 1,
    activation = 'relu'
)(x)
x = tf.keras.layers.MaxPool2D(
    pool_size = 2,
    strides = 1,
    padding = 'valid'
)(x)
x = tf.keras.layers.Flatten()(x)
x = mann.layers.MaskedDense(256, activation = 'relu')(x)
x = mann.layers.MaskedDense(256, activation = 'relu')(x)
output_layer = mann.layers.MaskedDense(10, activation = 'softmax')(x)

# Create the model
model = tf.keras.Model(input_layer, output_layer)

2021-11-09 07:25:38.653338: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-09 07:25:38.654033: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


In [4]:
# Compile the model for training and to prepare for masking
model.compile(
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'],
    optimizer = 'adam'
)

# Mask (prune) the model using the MANN package
model = mann.utils.mask_model(
    model = model,              # The model to be pruned
    percentile = 80,            # The percentile to be masked, for example, if the value is 90, then 90% of weights will be masked
    method = 'gradients',       # The method to use to mask, either 'gradients' or 'magnitude'
    exclusive = True,           # Whether weight locations must be exclusive to each task
    x = digit_x_train[:1000],   # The input data (using a subset to calculate gradients)
    y = digit_y_train[:1000]    # The expected outputs (using a subset to calculate gradients)
)

# Recompile the model
model.compile(
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'],
    optimizer = 'adam'
)

In [5]:
# Show the sparsity of the model
model.layers[1].get_weights()[0]

array([[[[-0.        ,  0.        , -0.        , -0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,
          -0.        ,  0.        , -0.        ,  0.02906718,
           0.05429413,  0.        ,  0.        , -0.        ,
          -0.02267656, -0.        ,  0.        , -0.        ,
          -0.05906082, -0.12245995,  0.04115575,  0.        ,
           0.04951426,  0.        ,  0.        ,  0.        ,
          -0.        , -0.        , -0.        ,  0.        ]],

        [[-0.        ,  0.        , -0.        , -0.        ,
           0.        , -0.        ,  0.        ,  0.        ,
           0.        , -0.        ,  0.        ,  0.0459222 ,
           0.03733057, -0.        ,  0.05112445, -0.        ,
           0.05061556, -0.        , -0.        , -0.        ,
           0.1067455 ,  0.        , -0.00599049,  0.        ,
           0.02466581, -0.        , -0.0068549 ,  0.        ,
          -0.        , -0.        , -0.        ,  0.        ]],

  

In [6]:
# Fit the model on the first dataset
model.fit(
    digit_x_train,
    digit_y_train,
    batch_size = 128,
    epochs = 100,
    validation_split = 0.2,
    callbacks = [callback]
)
print(f'Digit Model Accuracy: {(model.predict(digit_x_test).argmax(axis = 1).flatten() == digit_y_test.flatten()).sum()/digit_y_test.shape[0]}')

2021-11-09 07:26:22.954960: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-09 07:26:22.957940: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/100


2021-11-09 07:26:23.170031: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


375/375 [==============================] - ETA: 0s - loss: 0.3538 - accuracy: 0.8902

2021-11-09 07:26:56.007482: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


375/375 [==============================] - 36s 94ms/step - loss: 0.3538 - accuracy: 0.8902 - val_loss: 0.1025 - val_accuracy: 0.9703
Epoch 2/100
375/375 [==============================] - 35s 93ms/step - loss: 0.0813 - accuracy: 0.9756 - val_loss: 0.0761 - val_accuracy: 0.9790
Epoch 3/100
375/375 [==============================] - 35s 94ms/step - loss: 0.0540 - accuracy: 0.9834 - val_loss: 0.0548 - val_accuracy: 0.9831
Epoch 4/100
375/375 [==============================] - 35s 92ms/step - loss: 0.0393 - accuracy: 0.9874 - val_loss: 0.0716 - val_accuracy: 0.9818
Epoch 5/100
375/375 [==============================] - 35s 93ms/step - loss: 0.0293 - accuracy: 0.9907 - val_loss: 0.0544 - val_accuracy: 0.9853
Epoch 6/100
375/375 [==============================] - 35s 94ms/step - loss: 0.0232 - accuracy: 0.9924 - val_loss: 0.0463 - val_accuracy: 0.9878


2021-11-09 07:29:54.027378: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Digit Model Accuracy: 0.9864


## Create the second model

The second model will have an identical architecture to the first, but it will be trained to perform MNIST fashion recognition instead.

In [7]:
# Create the input layer for the fashion task
input_layer = tf.keras.layers.Input(fashion_x_train.shape[1:])

# Create the convolutional blocks
x = mann.layers.MaskedConv2D(
    filters = 32,
    kernel_size = 3,
    padding = 'same',
    strides = 1,
    activation = 'relu'
)(input_layer)
x = mann.layers.MaskedConv2D(
    filters = 32,
    kernel_size = 3,
    padding = 'same',
    strides = 1,
    activation = 'relu'
)(x)
x = tf.keras.layers.MaxPool2D(
    pool_size = 2,
    strides = 1,
    padding = 'valid'
)(x)
x = mann.layers.MaskedConv2D(
    filters = 64,
    kernel_size = 3,
    padding = 'same',
    strides = 1,
    activation = 'relu'
)(x)
x = mann.layers.MaskedConv2D(
    filters = 64,
    kernel_size = 3,
    padding = 'same',
    strides = 1,
    activation = 'relu'
)(x)
x = tf.keras.layers.MaxPool2D(
    pool_size = 2,
    strides = 1,
    padding = 'valid'
)(x)
x = tf.keras.layers.Flatten()(x)
x = mann.layers.MaskedDense(256, activation = 'relu')(x)
x = mann.layers.MaskedDense(256, activation = 'relu')(x)
output_layer = mann.layers.MaskedDense(10, activation = 'softmax')(x)

# Create the model
model = tf.keras.Model(input_layer, output_layer)

In [8]:
# Compile the model for training and to prepare for masking
model.compile(
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'],
    optimizer = 'adam'
)

# Mask (prune) the model using the MANN package
model = mann.utils.mask_model(
    model = model,              # The model to be pruned
    percentile = 80,            # The percentile to be masked, for example, if the value is 90, then 90% of weights will be masked
    method = 'gradients',       # The method to use to mask, either 'gradients' or 'magnitude'
    exclusive = True,           # Whether weight locations must be exclusive to each task
    x = fashion_x_train[:1000], # The input data (using a subset to calculate gradients)
    y = fashion_y_train[:1000]  # The expected outputs (using a subset to calculate gradients)
)

# Recompile the model
model.compile(
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'],
    optimizer = 'adam'
)

In [9]:
# Fit the model on the second dataset
model.fit(
    fashion_x_train,
    fashion_y_train,
    batch_size = 128,
    epochs = 100,
    validation_split = 0.2,
    callbacks = [callback]
)
print(f'Fashion Model Accuracy: {(model.predict(fashion_x_test).argmax(axis = 1).flatten() == fashion_y_test.flatten()).sum()/fashion_y_test.shape[0]}')

Epoch 1/100


2021-11-09 07:30:04.183111: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


375/375 [==============================] - ETA: 0s - loss: 0.6690 - accuracy: 0.7523

2021-11-09 07:30:35.875719: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


375/375 [==============================] - 35s 93ms/step - loss: 0.6690 - accuracy: 0.7523 - val_loss: 0.4472 - val_accuracy: 0.8403
Epoch 2/100
375/375 [==============================] - 35s 92ms/step - loss: 0.3974 - accuracy: 0.8595 - val_loss: 0.3747 - val_accuracy: 0.8682
Epoch 3/100
375/375 [==============================] - 34s 92ms/step - loss: 0.3344 - accuracy: 0.8789 - val_loss: 0.3323 - val_accuracy: 0.8767
Epoch 4/100
375/375 [==============================] - 34s 92ms/step - loss: 0.2872 - accuracy: 0.8957 - val_loss: 0.2913 - val_accuracy: 0.8937
Epoch 5/100
375/375 [==============================] - 35s 92ms/step - loss: 0.2547 - accuracy: 0.9072 - val_loss: 0.2776 - val_accuracy: 0.8961
Epoch 6/100
375/375 [==============================] - 35s 92ms/step - loss: 0.2267 - accuracy: 0.9159 - val_loss: 0.2530 - val_accuracy: 0.9066
Epoch 7/100
375/375 [==============================] - 34s 92ms/step - loss: 0.1986 - accuracy: 0.9268 - val_loss: 0.2530 - val_accuracy: 0.91

2021-11-09 07:35:15.092436: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Fashion Model Accuracy: 0.8995


## Create the MANN

The third and final model we create here will be a multitask model (MANN) which performs both the MNIST digit recognition and the MNIST fashion recognition tasks.

In [10]:
# Train the Multitask Model
digit_input = tf.keras.layers.Input(digit_x_train.shape[1:])
fashion_input = tf.keras.layers.Input(fashion_x_train.shape[1:])

# Create the convolutional blocks
x = mann.layers.MultiMaskedConv2D(
    filters = 32,
    kernel_size = 3,
    padding = 'same',
    strides = 1,
    activation = 'relu'
)([digit_input, fashion_input])
x = mann.layers.MultiMaskedConv2D(
    filters = 32,
    kernel_size = 3,
    padding = 'same',
    strides = 1,
    activation = 'relu'
)(x)
x = mann.layers.MultiMaxPool2D(
    pool_size = 2,
    strides = 1,
    padding = 'valid'
)(x)
x = mann.layers.MultiMaskedConv2D(
    filters = 64,
    kernel_size = 3,
    padding = 'same',
    strides = 1,
    activation = 'relu'
)(x)
x = mann.layers.MultiMaskedConv2D(
    filters = 64,
    kernel_size = 3,
    padding = 'same',
    strides = 1,
    activation = 'relu'
)(x)
x = mann.layers.MultiMaxPool2D(
    pool_size = 2,
    strides = 1,
    padding = 'valid'
)(x)

# SelectorLayer for the first task
sel1 = mann.layers.SelectorLayer(0)(x)
digit_flatten = tf.keras.layers.Flatten()(sel1)

# SelectorLayer for the second task
sel2 = mann.layers.SelectorLayer(1)(x)
fashion_flatten = tf.keras.layers.Flatten()(sel2)

x = mann.layers.MultiMaskedDense(256, activation = 'relu')([digit_flatten, fashion_flatten])
x = mann.layers.MultiMaskedDense(256, activation = 'relu')(x)
output_layer = mann.layers.MultiMaskedDense(10, activation = 'softmax')(x)

# Create the model
model = tf.keras.Model([digit_input, fashion_input], output_layer)

In [11]:
# Perform masking
model.compile(
    loss = 'sparse_categorical_crossentropy',
    
    metrics = ['accuracy'],
    optimizer = 'adam'
)
model = mann.utils.mask_model(
    model,
    80,
    method = 'gradients',
    exclusive = True,
    x = [digit_x_train[:1000], fashion_x_train[:1000]],
    y = [digit_y_train[:1000], fashion_y_train[:1000]]
)
model.compile(
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'],
    optimizer = 'adam'
)

model.fit(
    [digit_x_train, fashion_x_train],
    [digit_y_train, fashion_y_train],
    epochs = 100,
    batch_size = 128,
    callbacks = [callback],
    validation_split = 0.2
)

Epoch 1/100


2021-11-09 07:35:24.650070: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


375/375 [==============================] - ETA: 0s - loss: 1.0392 - multi_masked_dense_2_loss: 0.3406 - multi_masked_dense_2_1_loss: 0.6986 - multi_masked_dense_2_accuracy: 0.8944 - multi_masked_dense_2_1_accuracy: 0.7496

2021-11-09 07:36:33.194804: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


375/375 [==============================] - 76s 201ms/step - loss: 1.0392 - multi_masked_dense_2_loss: 0.3406 - multi_masked_dense_2_1_loss: 0.6986 - multi_masked_dense_2_accuracy: 0.8944 - multi_masked_dense_2_1_accuracy: 0.7496 - val_loss: 0.5564 - val_multi_masked_dense_2_loss: 0.1030 - val_multi_masked_dense_2_1_loss: 0.4534 - val_multi_masked_dense_2_accuracy: 0.9681 - val_multi_masked_dense_2_1_accuracy: 0.8379
Epoch 2/100
375/375 [==============================] - 75s 200ms/step - loss: 0.4368 - multi_masked_dense_2_loss: 0.0758 - multi_masked_dense_2_1_loss: 0.3610 - multi_masked_dense_2_accuracy: 0.9768 - multi_masked_dense_2_1_accuracy: 0.8707 - val_loss: 0.3866 - val_multi_masked_dense_2_loss: 0.0650 - val_multi_masked_dense_2_1_loss: 0.3216 - val_multi_masked_dense_2_accuracy: 0.9797 - val_multi_masked_dense_2_1_accuracy: 0.8838
Epoch 3/100
375/375 [==============================] - 75s 199ms/step - loss: 0.3297 - multi_masked_dense_2_loss: 0.0484 - multi_masked_dense_2_1_lo

# Predict Using the MANN

Now that the MANN model has been trained, we can use it to get predictions just as we would a traditional model. In this case, a list of predictions are returned, with each index corresponding to the task.

In [12]:
digit_preds, fashion_preds = model.predict([digit_x_test, fashion_x_test])
digit_preds = digit_preds.argmax(axis = 1)
fashion_preds = fashion_preds.argmax(axis = 1)

print(f'Multitask Model Digit Accuracy: {(digit_preds.flatten() == digit_y_test.flatten()).sum()/digit_y_test.flatten().shape[0]}')
print(f'Multitask Model Fashion Accuracy: {(fashion_preds.flatten() == fashion_y_test.flatten()).sum()/fashion_y_test.flatten().shape[0]}')

2021-11-09 07:46:41.027169: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Multitask Model Digit Accuracy: 0.9883
Multitask Model Fashion Accuracy: 0.9164
